In [2]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

In [3]:
!pip install -qU qdrant-client pymupdf pandas

In [13]:
import os
import openai
import chainlit as cl
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Qdrant
from langchain.prompts import ChatPromptTemplate

from dotenv import load_dotenv
from operator import itemgetter
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable.config import RunnableConfig

#Load environment variables
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

#Load 10-K PDF and split into chunks
loader = PyMuPDFLoader (
    "./data/AirBNB10kfilingsq12024.pdf"
)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

documents = text_splitter.split_documents(documents)

#Load embeddings model - we'll use OpenAI's text-embedding-3-small
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

#Create QDrant vector store
qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="AirBNB10k",
)

#Create Retriever
retriever = qdrant_vector_store.as_retriever()

#Create Prompt Template
template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#Choose LLM - we'll use gpt-4o.
primary_llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

2024-06-20 23:18:02 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-20 23:18:03 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-20 23:18:04 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-20 23:18:05 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-20 23:18:06 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [14]:
retrieved_documents = retriever.invoke("What was the total value of 'Cash and cash equivalents' as of December 31, 2023?")

2024-06-20 23:18:10 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [15]:
for doc in retrieved_documents:
  print(doc)

page_content='Table of Contents\nAirbnb, Inc.\nNotes to Condensed Consolidated Financial Statements (unaudited)\nNote 3. Supplemental Financial Statement Information\nCash, Cash Equivalents, and Restricted Cash\nThe following table reconciles cash, cash equivalents, and restricted cash reported on the Company’s unaudited condensed consolidated balance sheets to the total amount\npresented in the unaudited condensed consolidated statements of cash flows (in millions):\nDecember 31,\n2023\nMarch 31,\n2024\nCash and cash equivalents\n$\n6,874\xa0 $\n7,829\xa0\nCash and cash equivalents included in funds receivable and amounts held on behalf of customers\n5,769\xa0\n8,665\xa0\nRestricted cash included in prepaids and other current assets\n24\xa0\n35\xa0\nTotal cash, cash equivalents, and restricted cash presented in the unaudited condensed consolidated statements of cash flows\n$\n12,667\xa0 $\n16,529\xa0\nSupplemental disclosures of balance sheet information\nSupplemental balance sheet in

In [16]:
primary_llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

retrieval_augmented_chain = (
        # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
        # "question" : populated by getting the value of the "question" key
        # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | prompt | primary_llm
    )

In [17]:
question = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"

result = retrieval_augmented_chain.invoke({"question" : question})

print(result["response"].content)

2024-06-20 23:18:25 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-06-20 23:18:26 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


TypeError: 'AIMessage' object is not subscriptable

In [20]:
print(result)

content="The total value of 'Cash and cash equivalents' as of December 31, 2023, was $6,874 million." response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 2129, 'total_tokens': 2156}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'stop', 'logprobs': None} id='run-1a2044fd-54bb-4f2d-bb88-cfafd050ee3c-0' usage_metadata={'input_tokens': 2129, 'output_tokens': 27, 'total_tokens': 2156}
